In [0]:

import time
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

# We want to be able to repeat the experiment so we fix to a random seed
np.random.seed(70)

# Let us loading data 
data=pd.read_csv("ASX200Daily.csv",usecols=['Close'])##usecols=['Date', 'Close'])
data = data.dropna()  # Drop all Nans
data = data.values  # Convert from DataFrame to Python Array  
                    # You need to make sure the data is type of float
                    # you may use data = data.astype('float32') if your data are integers

# Prepare data .....


In [0]:

""" Scaling ...
Neural networks normally work well with scaled data, especially when we use
the sigmoid or tanh activation function. It is a good practice to scale the
data to the range of 0-to-1. This can be easily done by using scikit-learn's 
MinMaxScaler 
"""
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

"""  Splitting ...
We are going to use a time window of k = 3, so we will split the time series as
   x_1, x_2, x_3,     x_4  [prediction]
   x_2, x_3, x_4,     x_5
   x_3, x_4, x_5,     x_6 
   ....
""" 
train_size = int(len(data)*0.7)   
test_size = len(data) - train_size 
Dtrain, Dtest = data[0:train_size,:], data[train_size:len(data),:]

time_window =15
Xtrain, Ytrain = [], []
for i in range(len(Dtrain) - time_window -1):
    Xtrain.append(Dtrain[i:(i+time_window), 0])   
    Ytrain.append(Dtrain[i+time_window, 0])      
Xtrain = np.array(Xtrain)      
Ytrain = np.array(Ytrain) 
 

Xtest, Ytest = [], []
for i in range(len(Dtest) - time_window -1):
    Xtest.append(Dtest[i:(i+time_window), 0])   
    Ytest.append(Dtest[i+time_window, 0])       
Xtest = np.array(Xtest)    
Ytest = np.array(Ytest) 


Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], time_window, 1))  
Xtest = np.reshape(Xtest, (Xtest.shape[0], time_window, 1))  



In [0]:
# Define our model .....
model = Sequential()


MyBatchSize = 1

model.add(LSTM(100, input_shape = (time_window,1), batch_size=MyBatchSize,
        return_sequences=False))   # Many-to-One model

# It seems without using batch_size = 1 here causes some problems, a bug?

model.add(Dropout(0.2))            # Impose sparsity as we have so many hidden neurons
# As we will have 100 outputs from LSTM at each time step, we will use a linear 
# layer to map them to a single "prediction" output
model.add(Dense(
        output_dim=1))
model.add(Activation("linear"))

# Compiling model for use
start = time.time()
model.compile(loss="mse", optimizer="Adadelta")
print("Compilation Time : ", time.time() - start)


# Training
model.fit(
	    Xtrain,
	    Ytrain,
	    batch_size=MyBatchSize,
	    nb_epoch=50,   # You increase this number
	    validation_split=0.05)

# Predicting
# make predictions
trainPredict = model.predict(Xtrain,batch_size=MyBatchSize)
testPredict = model.predict(Xtest,batch_size=MyBatchSize)
# invert predictions due to scaling
trainPredict = scaler.inverse_transform(trainPredict)
Ytrain = scaler.inverse_transform(Ytrain[:,np.newaxis])
testPredict = scaler.inverse_transform(testPredict)
Ytest = scaler.inverse_transform(Ytest[:,np.newaxis])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(Ytrain, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(Ytest, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))



/Users/ratnesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  app.launch_new_instance()
/Users/ratnesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Compilation Time :  0.03567790985107422
Train on 3189 samples, validate on 168 samples
Epoch 1/50
 453/3189 [===>..........................] - ETA: 46s - loss: 0.0055

KeyboardInterrupt: 

In [0]:
# Plotting results
# shift train predictions for plotting
trainPredictPlot = np.empty_like(data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[time_window:len(trainPredict)+time_window, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(time_window*2)+1:len(data)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(data), label='True Data')
plt.plot(trainPredictPlot, label='Train Prediction')
plt.plot(testPredictPlot, label='Test Prediction')
plt.legend()
plt.show()


In [0]:
## Overall Predictions 
Xall = [] 
predictions = [] 

for i in range(6):
    Xall = []
    for j in range(len(data) - time_window):
        Xall.append(data[j:(j+time_window), 0])   # pick up the section in time_window size
    Xall = np.array(Xall)    # Convert them from list to array  
    Xall = np.reshape(Xall, (Xall.shape[0], time_window, 1)) 
    
    allPredict = model.predict(Xall,batch_size=MyBatchSize)
    data=np.append(data,allPredict[-1]) 
    data=data[:,np.newaxis]  
    allPredict = scaler.inverse_transform(allPredict)
    predictions.append(allPredict[-1])
    

data=data[:len(data)-1]

allPredictPlot = np.empty_like(data)
allPredictPlot[:, :] = np.nan
allPredictPlot[time_window:, :] = allPredict

plt.figure()
plt.plot(scaler.inverse_transform(data), label='True Data')
plt.plot(allPredictPlot, label='One-Step Prediction') 
plt.legend()
plt.show()

